In [3]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

# Paths to dataset
yes_path = 'Brain MRI Images/yes'
no_path = 'Brain MRI Images/no'

def load_data(data_path, label, size=(128, 128)):
    images = []
    labels = []
    for img_file in os.listdir(data_path):
        if not img_file.lower().endswith(('.png', '.jpg', '.jpeg')):
            print(f"Skipping non-image file: {img_file}")
            continue
        img_path = os.path.join(data_path, img_file)
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        if img is None:
            print(f"Skipping invalid file: {img_path}")
            continue
        img = cv2.resize(img, size)
        images.append(img)
        labels.append(label)
    return np.array(images), np.array(labels)

# Load data from both folders
yes_images, yes_labels = load_data(yes_path, label=1)
no_images, no_labels = load_data(no_path, label=0)

# Combine and split data
X = np.concatenate([yes_images, no_images], axis=0)
y = np.concatenate([yes_labels, no_labels], axis=0)

# Normalize and expand dimensions for the model
X = X / 255.0
X = np.expand_dims(X, axis=-1)  # Shape becomes (samples, 128, 128, 1)

# Convert labels to categorical
y = to_categorical(y, num_classes=2)

# Split into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"Training set shape: {X_train.shape}")
print(f"Validation set shape: {X_val.shape}")

Skipping non-image file: .DS_Store
Skipping non-image file: .DS_Store
Training set shape: (188, 128, 128, 1)
Validation set shape: (48, 128, 128, 1)


In [4]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

def create_model(input_shape=(128, 128, 1)):
    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(128, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(2, activation='softmax'))
    
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Create the model
model = create_model()

# Print the model summary
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_3 (Conv2D)               │ (None, 126, 126, 32)   │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 63, 63, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 61, 61, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 30, 30, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 28, 28, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (None, 14, 14, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 128)            │     3,211,392 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 2)              │           258 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,304,322 (12.60 MB)

 Trainable params: 3,304,322 (12.60 MB)

 Non-trainable params: 0 (0.00 B)

In [5]:
history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=50,
    batch_size=32
)

Epoch 1/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 158ms/step - accuracy: 0.6064 - loss: 0.6351 - val_accuracy: 0.7292 - val_loss: 0.5516
Epoch 2/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 128ms/step - accuracy: 0.7719 - loss: 0.5240 - val_accuracy: 0.7500 - val_loss: 0.4763
Epoch 3/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 128ms/step - accuracy: 0.7880 - loss: 0.5075 - val_accuracy: 0.8125 - val_loss: 0.4383
Epoch 4/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 128ms/step - accuracy: 0.7227 - loss: 0.5348 - val_accuracy: 0.8125 - val_loss: 0.4357
Epoch 5/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 128ms/step - accuracy: 0.8209 - loss: 0.4064 - val_accuracy: 0.8125 - val_loss: 0.3822
Epoch 6/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 124ms/step - accuracy: 0.8268 - loss: 0.3956 - val_accuracy: 0.8333 - val_loss: 0.3864
Epoch 7/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 125ms/step - accuracy: 0.7953 - loss: 0.4243 - val_accuracy: 0.8958 - val_loss: 0.3155
Epoch 8/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 137ms/step - accuracy: 0.8401 - loss: 0.2998 - val_accuracy: 0.9375 - val_loss:

In [6]:
# Evaluate the model
loss, accuracy = model.evaluate(X_val, y_val)
print(f"Validation Loss: {loss}")
print(f"Validation Accuracy: {accuracy}")

# Save the trained model
model.save('brain_mri_tumor_classifier3.h5')

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - accuracy: 0.9236 - loss: 0.5275


Validation Loss: 0.5748909115791321
Validation Accuracy: 0.9166666865348816
